<h1> Parameter Tunning 
    
    Look at n_neurons & n_hidden layers

# Importing libraries:

In [37]:
!pip install pandas
!pip install sklearn

import pandas as pd;
import numpy as np;
import sklearn

# Import data; set X and y; fill nan values and split in test and training  data:

In [38]:
# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# for later use
features_num = 15

# 2018 data
data = data.loc[data.index > 2018070000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# fill nan values
data.fillna(method = 'ffill', inplace = True)

from sklearn.model_selection import train_test_split

# divide data into train and test 
data_train, data_test = train_test_split(
         data, test_size = 0.15, shuffle=False)

# Apply feature scaling:

In [39]:
from sklearn.preprocessing import MinMaxScaler

# data scaling  (including offer (y))
sc_X = MinMaxScaler()
data_train = sc_X.fit_transform(data_train)
data_test = sc_X.transform(data_test)

# Importing packages:

In [40]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping

mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []
hist_list = []
y_pred_list = []
prediction_list = []
time_count = []

# Prepare of data according to LSTM needs,  create regressor & tune:

In [ ]:
# parameters
steps = 96
#n_hidden = 1
#units = 100
batch_size = 96

from sklearn.model_selection import ParameterGrid

# no hidden or one hidden layer
parameters = {'n_hidden': [1, 2],
              'units': [50, 100, 150, 200]}

all_param = ParameterGrid(parameters)

# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

# divide features and labels
X_train = data_train[:, 0:15] 
y_train = data_train[:, -1]
X_test = data_test[:, 0:15] 
y_test = data_test[:, -1] 

# divide data into train and test 
X_train, X_val, y_train, y_val = train_test_split(
         X_train, y_train, test_size = 0.15, shuffle=False)

# put data into correct shape
X_train, y_train = split_data(X_train, y_train, steps)
X_test, y_test = split_data(X_test, y_test, steps)
X_val, y_val = split_data(X_val, y_val, steps)

X_train = cut_data(X_train, batch_size)
y_train = cut_data(y_train, batch_size)
X_test = cut_data(X_test, batch_size)
y_test = cut_data(y_test, batch_size)
X_val = cut_data(X_val, batch_size)
y_val = cut_data(y_val, batch_size)

# design the LSTM
def regressor_tunning(kernel_initializer = 'he_normal',
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    if n_hidden == 0:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    else:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       return_sequences = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
        model.add(LSTM(units = units, 
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = 'Adamax')
    return model

# inverse of test set should not be inside the loop 
y_test = (y_test * sc_X.data_range_[15]) + (sc_X.data_min_[15])

for i in range(len(all_param)):
    
    units = all_param[i]['units']
    n_hidden = all_param[i]['n_hidden']
    
    model = regressor_tunning()

    # fitting the LSTM to the training set
    history = model.fit(X_train,
                        y_train, 
                        batch_size = batch_size, 
                        epochs = 180,
                        shuffle = False, 
                        validation_data = (X_val, y_val))
    
    model.reset_states()
    
    # make new predicitons with test set
    y_pred = model.predict(X_test, batch_size = batch_size)
    
    # prices col = 15
    y_pred = (y_pred * sc_X.data_range_[15]) + (sc_X.data_min_[15])

    y_pred_list.append(y_pred)
    
    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mae_error = mae(y_test, y_pred)
    
    rmse_gen.append(rmse_error)
    mae_gen.append(mae_error)
    
    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])
    
    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values
    
    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)

    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mae_spike = mae(y_test_spike, y_pred_spike)
    
    rmse_spi.append(rmse_spike)
    mae_spi.append(mae_spike)
    
    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================
    
    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)
    
    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mae_normal = mae(y_test_normal, y_pred_normal)
    
    rmse_nor.append(rmse_normal)
    mae_nor.append(mae_normal)

Epoch 1/180
65/65 [==============================] - 23s 351ms/step - loss: 0.1158 - mse: 0.1158 - mae: 0.2701 - val_loss: 0.0033 - val_mse: 0.0033 - val_mae: 0.0425
Epoch 2/180
65/65 [==============================] - 20s 311ms/step - loss: 0.0632 - mse: 0.0632 - mae: 0.2009 - val_loss: 0.0021 - val_mse: 0.0021 - val_mae: 0.0329
Epoch 3/180
65/65 [==============================] - 19s 286ms/step - loss: 0.0409 - mse: 0.0409 - mae: 0.1608 - val_loss: 0.0014 - val_mse: 0.0014 - val_mae: 0.0242
Epoch 4/180
65/65 [==============================] - 19s 297ms/step - loss: 0.0281 - mse: 0.0281 - mae: 0.1328 - val_loss: 0.0013 - val_mse: 0.0013 - val_mae: 0.0199
Epoch 5/180
65/65 [==============================] - 19s 297ms/step - loss: 0.0203 - mse: 0.0203 - mae: 0.1122 - val_loss: 0.0012 - val_mse: 0.0012 - val_mae: 0.0234
Epoch 6/180
65/65 [==============================] - 19s 291ms/step - loss: 0.0152 - mse: 0.0152 - mae: 0.0974 - val_loss: 0.0014 - val_mse: 0.0014 - val_mae: 0.0223
Epoc

65/65 [==============================] - 14s 208ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0197 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0186
Epoch 51/180
65/65 [==============================] - 14s 209ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0194 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0189
Epoch 52/180
65/65 [==============================] - 13s 203ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0193 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0186
Epoch 53/180
65/65 [==============================] - 14s 209ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0193 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0186
Epoch 54/180
65/65 [==============================] - 14s 209ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0190 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0189
Epoch 55/180
65/65 [==============================] - 14s 210ms/step - loss: 9.9157e-04 - mse: 9.9157e-04 - mae: 0.0189 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0186
Epo

65/65 [==============================] - 15s 237ms/step - loss: 9.1504e-04 - mse: 9.1504e-04 - mae: 0.0178 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0185
Epoch 98/180
65/65 [==============================] - 15s 232ms/step - loss: 9.1416e-04 - mse: 9.1416e-04 - mae: 0.0179 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0186
Epoch 99/180
65/65 [==============================] - 15s 226ms/step - loss: 9.1485e-04 - mse: 9.1485e-04 - mae: 0.0178 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0186
Epoch 100/180
65/65 [==============================] - 15s 231ms/step - loss: 9.1348e-04 - mse: 9.1348e-04 - mae: 0.0178 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0187
Epoch 101/180
65/65 [==============================] - 15s 223ms/step - loss: 9.1149e-04 - mse: 9.1149e-04 - mae: 0.0178 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0186
Epoch 102/180
65/65 [==============================] - 15s 230ms/step - loss: 9.1474e-04 - mse: 9.1474e-04 - mae: 0.0178 - val_loss: 0.0

Epoch 143/180
65/65 [==============================] - 15s 223ms/step - loss: 8.6139e-04 - mse: 8.6139e-04 - mae: 0.0169 - val_loss: 8.4613e-04 - val_mse: 8.4613e-04 - val_mae: 0.0182
Epoch 144/180
65/65 [==============================] - 15s 226ms/step - loss: 8.6587e-04 - mse: 8.6587e-04 - mae: 0.0170 - val_loss: 8.6869e-04 - val_mse: 8.6869e-04 - val_mae: 0.0187
Epoch 145/180
65/65 [==============================] - 15s 237ms/step - loss: 8.5755e-04 - mse: 8.5755e-04 - mae: 0.0168 - val_loss: 8.7831e-04 - val_mse: 8.7831e-04 - val_mae: 0.0186
Epoch 146/180
65/65 [==============================] - 15s 225ms/step - loss: 8.5980e-04 - mse: 8.5980e-04 - mae: 0.0168 - val_loss: 8.4720e-04 - val_mse: 8.4720e-04 - val_mae: 0.0179
Epoch 147/180
65/65 [==============================] - 14s 223ms/step - loss: 8.5843e-04 - mse: 8.5843e-04 - mae: 0.0168 - val_loss: 8.6682e-04 - val_mse: 8.6682e-04 - val_mae: 0.0184
Epoch 148/180
65/65 [==============================] - 14s 221ms/step - loss: 8.

65/65 [==============================] - 42s 652ms/step - loss: 0.0096 - mse: 0.0096 - mae: 0.0764 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0199
Epoch 9/180
65/65 [==============================] - 43s 663ms/step - loss: 0.0080 - mse: 0.0080 - mae: 0.0704 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0229
Epoch 10/180
65/65 [==============================] - 44s 673ms/step - loss: 0.0068 - mse: 0.0068 - mae: 0.0643 - val_loss: 0.0012 - val_mse: 0.0012 - val_mae: 0.0265
Epoch 11/180
65/65 [==============================] - 41s 626ms/step - loss: 0.0055 - mse: 0.0055 - mae: 0.0578 - val_loss: 0.0015 - val_mse: 0.0015 - val_mae: 0.0334
Epoch 12/180
65/65 [==============================] - 38s 591ms/step - loss: 0.0047 - mse: 0.0047 - mae: 0.0528 - val_loss: 0.0015 - val_mse: 0.0015 - val_mae: 0.0326
Epoch 13/180
65/65 [==============================] - 42s 645ms/step - loss: 0.0042 - mse: 0.0042 - mae: 0.0499 - val_loss: 0.0013 - val_mse: 0.0013 - val_mae: 0.0292
Epoch 14/180

65/65 [==============================] - 45s 688ms/step - loss: 9.9083e-04 - mse: 9.9083e-04 - mae: 0.0190 - val_loss: 9.5340e-04 - val_mse: 9.5340e-04 - val_mae: 0.0196
Epoch 56/180
65/65 [==============================] - 46s 700ms/step - loss: 9.8669e-04 - mse: 9.8669e-04 - mae: 0.0189 - val_loss: 9.4625e-04 - val_mse: 9.4625e-04 - val_mae: 0.0196
Epoch 57/180
65/65 [==============================] - 46s 711ms/step - loss: 9.8658e-04 - mse: 9.8658e-04 - mae: 0.0187 - val_loss: 9.5155e-04 - val_mse: 9.5155e-04 - val_mae: 0.0195
Epoch 58/180
65/65 [==============================] - 45s 689ms/step - loss: 9.7132e-04 - mse: 9.7132e-04 - mae: 0.0186 - val_loss: 9.4393e-04 - val_mse: 9.4393e-04 - val_mae: 0.0189
Epoch 59/180
65/65 [==============================] - 47s 728ms/step - loss: 9.6564e-04 - mse: 9.6564e-04 - mae: 0.0184 - val_loss: 9.4476e-04 - val_mse: 9.4476e-04 - val_mae: 0.0190
Epoch 60/180
65/65 [==============================] - 40s 611ms/step - loss: 9.6176e-04 - mse: 9.6

Epoch 100/180
65/65 [==============================] - 41s 637ms/step - loss: 8.6611e-04 - mse: 8.6611e-04 - mae: 0.0168 - val_loss: 9.6934e-04 - val_mse: 9.6934e-04 - val_mae: 0.0173
Epoch 101/180
65/65 [==============================] - 40s 616ms/step - loss: 8.6857e-04 - mse: 8.6857e-04 - mae: 0.0168 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0195
Epoch 102/180
65/65 [==============================] - 42s 644ms/step - loss: 8.6352e-04 - mse: 8.6352e-04 - mae: 0.0167 - val_loss: 9.7369e-04 - val_mse: 9.7369e-04 - val_mae: 0.0175
Epoch 103/180
65/65 [==============================] - 43s 656ms/step - loss: 8.6077e-04 - mse: 8.6077e-04 - mae: 0.0167 - val_loss: 9.8669e-04 - val_mse: 9.8669e-04 - val_mae: 0.0179
Epoch 104/180
65/65 [==============================] - 45s 687ms/step - loss: 8.5904e-04 - mse: 8.5904e-04 - mae: 0.0167 - val_loss: 9.9508e-04 - val_mse: 9.9508e-04 - val_mae: 0.0180
Epoch 105/180
65/65 [==============================] - 39s 608ms/step - loss: 8.5496e-04

65/65 [==============================] - 41s 638ms/step - loss: 8.0501e-04 - mse: 8.0501e-04 - mae: 0.0159 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0191
Epoch 146/180
65/65 [==============================] - 40s 613ms/step - loss: 8.0143e-04 - mse: 8.0143e-04 - mae: 0.0158 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0183
Epoch 147/180
65/65 [==============================] - 39s 608ms/step - loss: 7.9866e-04 - mse: 7.9866e-04 - mae: 0.0158 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0183
Epoch 148/180
65/65 [==============================] - 38s 586ms/step - loss: 7.9063e-04 - mse: 7.9063e-04 - mae: 0.0158 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0183
Epoch 149/180
65/65 [==============================] - 37s 574ms/step - loss: 7.9625e-04 - mse: 7.9625e-04 - mae: 0.0159 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0193
Epoch 150/180
65/65 [==============================] - 37s 573ms/step - loss: 7.8431e-04 - mse: 7.8431e-04 - mae: 0.0157 - val_loss: 0

In [ ]:
results = pd.DataFrame({'all_param':all_param,
                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})

results.to_csv('Results_LSTM_2_n_neurons_n_hidden.csv')
results

In [ ]:
results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})

def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)